In [21]:
import torch

# quantize_per_tensor&dequantize

In [22]:
# 1. 创建一个普通的浮点张量
original_float = torch.tensor([-1.52, 0.01, 1.501, 3.0001])

In [23]:
# 2. 定义量化参数
scale = 0.5
zero_point = 0
dtype = torch.qint8

In [24]:
# 3. 量化：将 float32 -> qint8
# 公式：q = round(x / scale + zero_point)
q_tensor = torch.quantize_per_tensor(original_float, scale, zero_point, dtype)

print(f"量化后的张量 (int8): {q_tensor}")
print(f"存储的整数值: {q_tensor.int_repr()}")  # 查看底层的 int8 数值
print(f"存储的 scale: {q_tensor.q_scale()}")  # 查看存储的 scale
print(f"存储的 zero_point: {q_tensor.q_zero_point()}")  # 查看存储的 zero_point

量化后的张量 (int8): tensor([-1.5000,  0.0000,  1.5000,  3.0000], size=(4,), dtype=torch.qint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.5, zero_point=0)
存储的整数值: tensor([-3,  0,  3,  6], dtype=torch.int8)
存储的 scale: 0.5
存储的 zero_point: 0


In [25]:
dir(q_tensor)

['H',
 'T',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__red

In [26]:
# 4. 反量化：将 qint8 -> float32
# 公式：x = (q - zero_point) * scale
dequantized_tensor = torch.dequantize(q_tensor)
# 或者使用对象方法: q_tensor.dequantize()

print(f"反量化后的张量 (float32): {dequantized_tensor}")

反量化后的张量 (float32): tensor([-1.5000,  0.0000,  1.5000,  3.0000])


In [27]:
# 5. 观察误差
# 注意：反量化后的值可能与原始值不完全相等，因为量化是“有损”的
error = original_float - dequantized_tensor
print(f"精度损失: {error}")

精度损失: tensor([-2.0000e-02,  1.0000e-02,  1.0000e-03,  9.9897e-05])


# quantize_per_channel&dequantize

In [28]:
# 1. 准备数据
# 模拟一个 (4, 4) 的权重矩阵
# 假设有 4 个输出通道 (axis 0)，每个通道有 4 个输入特征
# 我们故意让不同通道的数值范围差异很大，以体现 Per-Channel 的优势
original_weights = torch.tensor(
    [
        [-0.1, 0.0, 0.1, 0.2],  # 通道 0: 数值很小
        [-100, -50, 0, 50],  # 通道 1: 数值很大
        [-110, -60, 20, 40],  # 通道 2: 数值很大
        [-5.0, -2.5, 2.5, 5.0],  # 通道 3: 数值中等
        [0.0, 0.0, 0.0, 0.0],  # 通道 4: 全是 0
    ],
    dtype=torch.float32,
)
original_weights

tensor([[-1.0000e-01,  0.0000e+00,  1.0000e-01,  2.0000e-01],
        [-1.0000e+02, -5.0000e+01,  0.0000e+00,  5.0000e+01],
        [-1.1000e+02, -6.0000e+01,  2.0000e+01,  4.0000e+01],
        [-5.0000e+00, -2.5000e+00,  2.5000e+00,  5.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]])

In [29]:
# 2. 定义量化参数 (Per Channel)
# 因为有 4 个通道 (dim=0)，我们需要 4 个 scale 和 4 个 zero_point
# axis=0 表示我们将沿着第 0 维（行）应用不同的量化参数
channel_axis = 0

# 手动定义 scale (通常这是通过观察数据 min/max 计算出来的，这里为了演示手动指定)
# 通道 0 范围小 -> scale 小
# 通道 1 范围大 -> scale 大
scales = torch.tensor([0.002, 0.5, 0.6, 0.05, 1.0])

# 定义 zero_points (int8 范围是 -128 到 127)
zero_points = torch.tensor([0, 0, 0, 0, 0])

# 指定量化数据类型，通常权重使用 qint8 (有符号 8 位整数)
dtype = torch.qint8

In [30]:
# 3. 执行 Per-Channel 量化
# quantize_per_channel(input, scales, zero_points, axis, dtype)
q_weights = torch.quantize_per_channel(
    original_weights, scales, zero_points, channel_axis, dtype
)

print(f"量化后的张量 (qint8):\n{q_weights}")
print(f"底层存储的整数值 (int_repr):\n{q_weights.int_repr()}")
print("-" * 20)
print(f"查看每个通道的 Scale: {q_weights.q_per_channel_scales()}")
print(f"查看每个通道的 Zero Point: {q_weights.q_per_channel_zero_points()}")
print("-" * 20)

量化后的张量 (qint8):
tensor([[ -0.1000,   0.0000,   0.1000,   0.2000],
        [-64.0000, -50.0000,   0.0000,  50.0000],
        [-76.8000, -60.0000,  19.8000,  40.2000],
        [ -5.0000,  -2.5000,   2.5000,   5.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000]], size=(5, 4),
       dtype=torch.qint8, quantization_scheme=torch.per_channel_affine,
       scale=tensor([0.0020, 0.5000, 0.6000, 0.0500, 1.0000], dtype=torch.float64),
       zero_point=tensor([0, 0, 0, 0, 0]), axis=0)
底层存储的整数值 (int_repr):
tensor([[ -50,    0,   50,  100],
        [-128, -100,    0,  100],
        [-128, -100,   33,   67],
        [-100,  -50,   50,  100],
        [   0,    0,    0,    0]], dtype=torch.int8)
--------------------
查看每个通道的 Scale: tensor([0.0020, 0.5000, 0.6000, 0.0500, 1.0000], dtype=torch.float64)
查看每个通道的 Zero Point: tensor([0, 0, 0, 0, 0])
--------------------


In [31]:
# 4. 执行反量化 (Dequantize)
# dequantize 会自动读取张量内部存储的 per_channel 信息进行解码
# 不需要再次传入 scales 或 zero_points
recovered_weights = torch.dequantize(q_weights)
# 也可以写成: recovered_weights = q_weights.dequantize()
recovered_weights

tensor([[ -0.1000,   0.0000,   0.1000,   0.2000],
        [-64.0000, -50.0000,   0.0000,  50.0000],
        [-76.8000, -60.0000,  19.8000,  40.2000],
        [ -5.0000,  -2.5000,   2.5000,   5.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000]])

In [32]:
# 5. 验证误差
error = original_weights - recovered_weights
error

# 验证逻辑：反量化值 = (int_val - zp) * scale
# 以通道 1 的第一个元素为例: 原始 -100, scale 0.5, zp 0
# int_val = -100 / 0.5 = -200 -> clamp 到 -128 (int8下限)
# dequant = -128 * 0.5 = -64
# 误差 = -100 - (-64) = -36 (这是因为 range 设置不合理导致的截断，仅作演示)

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-3.6000e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-3.3200e+01,  3.8147e-06,  2.0000e-01, -2.0000e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]])